In [2]:
!pip install tweet-preprocessor
!pip install transformers
!pip install sentence_transformers
!pip install ranky
!pip install keybert
!pip install pipreqs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=eae89cd6b43b10c2dfa0492709126487da3cd6f6ba0e00517d8de11b9e0326f7
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


## Imports

In [3]:
import snscrape
import preprocessor as p
import re
import nltk
import numpy as np
import pandas as pd
import transformers
import torch
import string
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
# from py_rankaggregation import CombANZ
import ranky as rk
from keybert import KeyBERT
import spacy

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Preprocessing

In [6]:
# download the necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
# Preprocessing
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Removing URLs, retweets, mentions, stopwords
    rtwords = ['rt', 'RT', 'retweet', 'Retweet', 'RETWEET']
    stops = stopwords.words('english') + rtwords
    
    mess = re.sub(r"http\S+", "", mess)
    mess = re.sub("@[A-Za-z0-9_]+","", mess)
    mess = re.sub("#","", mess)
    
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # # define the tokenizer, stemmer, and lemmatizer
    stop_words = set(stopwords.words('english'))
    # stemmer = SnowballStemmer('english')
    # lemmatizer = WordNetLemmatizer()

    # convert to lowercase
    tweet = nopunc.lower()
    # tokenize the tweet
    # tokens = word_tokenize(tweet)
    # remove stop words
    # tokens = [token for token in tokens if token not in stop_words]
    # # stem the tokens
    # tokens = [stemmer.stem(token) for token in tokens]
    # # lemmatize the tokens
    # tokens = [lemmatizer.lemmatize(token) for token in tokens]

    doc = nlp(tweet)
    doc = [token for token in doc if token not in stop_words]
    
    # Lemmatize the tokens
    lemmas = [token.lemma_ for token in doc]
    
    # Join the lemmas back into a single string
    processed_tweet = ' '.join(lemmas)

    return processed_tweet

## Loading the dataset

In [15]:
NEP_tweets = pd.read_csv('/content/drive/MyDrive/SMAI-project/data/NEP_2020_english_tweet.csv')
# NEP_tweets = pd.read_csv('NEP_2020_english_tweet.csv')

In [16]:
NEP_tweets

,Tweet_ID,Author_ID,Date_of_tweet,Tweet,Likes_on_tweet,User_handle,Tweet_link
0,0,60483175,2020-08-04 23:31:59+00:00,Historically #TamilNadu has been a state with ...,1,venkattcv,https://twitter.com/venkattcv/status/129079263...
1,1,1178900491595636736,2020-08-04 23:31:56+00:00,#HelloStrangerEP6 #HarrisonFord #GeorgeFloyd #...,0,BethEYoung3,https://twitter.com/BethEYoung3/status/1290792...
2,2,1072481505090318341,2020-08-04 22:22:30+00:00,@cbseindia29 @PMOIndia @narendramodi @DrRPNish...,1,PatelPrerak18,https://twitter.com/PatelPrerak18/status/12907...
3,3,583518901,2020-08-04 22:20:56+00:00,@PMOIndia’s anti-democratic #NEP2020 was 400+ ...,1,kchongpacrim,https://twitter.com/kchongpacrim/status/129077...
4,4,36339871,2020-08-04 21:01:14+00:00,The latest The Nursing Education Daily! http:/...,0,SkyscapeInc,https://twitter.com/SkyscapeInc/status/1290754...
...,...,...,...,...,...,...,...
18235,18235,2784359899,2020-08-05 01:06:05+00:00,First it was #Rafale Then #NEP2020 Tomorrow #R...,0,indhiranmagan,https://twitter.com/indhiranmagan/status/12908...
18236,18236,3309241736,2020-08-05 01:05:29+00:00,@KVishReddy sir would like to hear your remark...,0,mahiteja022,https://twitter.com/mahiteja022/status/1290816...
18237,18237,301309289,2020-08-05 01:02:31+00:00,While 6% of #GDP Announced 4 #NEP2020 is a #Ju...,0,mani24sjcpl,https://twitter.com/mani24sjcpl/status/1290815...
18238,18238,290922714,2020-08-05 00:49:39+00:00,Retrograde mindset of the political parties in...,1,bharvaz,https://twitter.com/bharvaz/status/12908121763...


In [17]:
NEP_tweets = NEP_tweets[['Tweet_ID', 'Date_of_tweet', 'Tweet', 'User_handle']]
NEP_tweets

,Tweet_ID,Date_of_tweet,Tweet,User_handle
0,0,2020-08-04 23:31:59+00:00,Historically #TamilNadu has been a state with ...,venkattcv
1,1,2020-08-04 23:31:56+00:00,#HelloStrangerEP6 #HarrisonFord #GeorgeFloyd #...,BethEYoung3
2,2,2020-08-04 22:22:30+00:00,@cbseindia29 @PMOIndia @narendramodi @DrRPNish...,PatelPrerak18
3,3,2020-08-04 22:20:56+00:00,@PMOIndia’s anti-democratic #NEP2020 was 400+ ...,kchongpacrim
4,4,2020-08-04 21:01:14+00:00,The latest The Nursing Education Daily! http:/...,SkyscapeInc
...,...,...,...,...
18235,18235,2020-08-05 01:06:05+00:00,First it was #Rafale Then #NEP2020 Tomorrow #R...,indhiranmagan
18236,18236,2020-08-05 01:05:29+00:00,@KVishReddy sir would like to hear your remark...,mahiteja022
18237,18237,2020-08-05 01:02:31+00:00,While 6% of #GDP Announced 4 #NEP2020 is a #Ju...,mani24sjcpl
18238,18238,2020-08-05 00:49:39+00:00,Retrograde mindset of the political parties in...,bharvaz


In [18]:
raw_tweets = NEP_tweets['Tweet'].tolist()
raw_tweets

["Historically #TamilNadu has been a state with progressive #socialpolicies &amp; high state/private investments in #education. Not without pitfalls but one of the best in India. If states were not involved in framing #NEP2020 as alleged here,I'm very surprised. ",
 '#HelloStrangerEP6 #HarrisonFord #GeorgeFloyd #JiChangWook #DishaSalian #footballindex #HappyEXOLDay #JusticeForSushant #KimSooHyun #KimSooHyun #GalaxyUnpacked #NitishKumar #NEP2020 #HappyEXOLDay #groth#Beauty#Beautyful#Enjoy Feeling#vasu',
 '@cbseindia29 @PMOIndia @narendramodi @DrRPNishank @ABPNews #NEP2020 is trending with its all new features of modern learning for brighter future but most of the students are behind the national entrance exams (JEE,NEET,SAT)etc. So how can NEP2020 will help us out in this exams',
 '@PMOIndia’s anti-democratic #NEP2020 was 400+ pgs in 2019 draft, and somehow 66 pgs after revisions. I want whoever cut that policy down to less that’s ¼ of its original size to read everything I’ve ever wrot

In [19]:
NEP_tweets['Tweet'] = NEP_tweets['Tweet'].apply(text_process)

<ipython-input-19-4a8997dd7701>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NEP_tweets['Tweet'] = NEP_tweets['Tweet'].apply(text_process)


In [20]:
NEP_tweets

,Tweet_ID,Date_of_tweet,Tweet,User_handle
0,0,2020-08-04 23:31:59+00:00,historically tamilnadu have be a state with pr...,venkattcv
1,1,2020-08-04 23:31:56+00:00,hellostrangerep6 harrisonford georgefloyd jich...,BethEYoung3
2,2,2020-08-04 22:22:30+00:00,nep2020 be trend with its all new featur...,PatelPrerak18
3,3,2020-08-04 22:20:56+00:00,’s antidemocratic nep2020 be 400 pgs in 2019 d...,kchongpacrim
4,4,2020-08-04 21:01:14+00:00,the late the nursing education daily thank t...,SkyscapeInc
...,...,...,...,...
18235,18235,2020-08-05 01:06:05+00:00,first it be rafale then nep2020 tomorrow ramma...,indhiranmagan
18236,18236,2020-08-05 01:05:29+00:00,sir would like to hear your remark on newedu...,mahiteja022
18237,18237,2020-08-05 01:02:31+00:00,while 6 of gdp announce 4 nep2020 be a jumla c...,mani24sjcpl
18238,18238,2020-08-05 00:49:39+00:00,retrograde mindset of the political party in t...,bharvaz


In [21]:
tweets = NEP_tweets['Tweet'].tolist()
tweets

['historically tamilnadu have be a state with progressive socialpolicie amp high stateprivate investment in education not without pitfall but one of the good in india if state be not involve in frame nep2020 as allege hereim very surprised',
 'hellostrangerep6 harrisonford georgefloyd jichangwook dishasalian footballindex happyexolday justiceforsushant kimsoohyun kimsoohyun galaxyunpacke nitishkumar nep2020 happyexolday grothbeautybeautyfulenjoy feelingvasu',
 '      nep2020 be trend with its all new feature of modern learning for bright future but most of the student be behind the national entrance exam jeeneetsatetc so how can nep2020 will help we out in this exam',
 '’s antidemocratic nep2020 be 400 pgs in 2019 draft and somehow 66 pgs after revision I want whoever cut that policy down to less that ’ ¼ of its original size to read everything I ’ve ever write',
 'the late the nursing education daily   thank to     education nep2020',
 'with rafale india have strengthen its defence ma

## Get tri-grams from each tweet

In [22]:
def get_ngrams(sentence, n):
    words = sentence.split()
    phrases = []
    for i in range(len(words)-(n-1)):
        phrase = " ".join(words[i:i+n])
        phrases.append(phrase)
    return phrases

In [23]:
# get n-grams for each tweet
ngrams = []
for tweet in tweets:
    ngrams.append(get_ngrams(tweet, 3))

# flatten list of lists
ngrams = [item for sublist in ngrams for item in sublist]
print(len(ngrams))

# get unique n-grams
unique_ngrams = list(set(ngrams))
print(len(unique_ngrams))

465668
122241


## Sentence Transformer Models

In [24]:
# Initializing the sentence transformer models
xlm_dist = SentenceTransformer('sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1')

distilbert = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-tas-b')

para_minilm = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

## Getting the embeddings of tweets and tri-grams

In [26]:
xlm_embeddings = xlm_dist.encode(tweets, convert_to_tensor=True, show_progress_bar=True)
print("\nXLM-Roberta embeddings of tweets:")
print(xlm_embeddings)

Batches:   0%|          | 0/570 [00:00<?, ?it/s]


XLM-Roberta embeddings of tweets:
tensor([[ 0.0309, -0.0085, -0.1028,  ...,  0.2689,  0.3677, -0.0713],
        [ 0.1520,  0.2277,  0.3146,  ...,  0.1624,  0.3976,  0.1051],
        [ 0.2715, -0.0573, -0.1566,  ...,  0.2717,  0.1956,  0.2781],
        ...,
        [ 0.1808,  0.1741,  0.0191,  ...,  0.6070,  0.3153, -0.1778],
        [ 0.0558,  0.1603, -0.1566,  ...,  0.3463,  0.1659,  0.2418],
        [ 0.2237,  0.1674, -0.1885,  ...,  0.6081,  0.1902,  0.1672]],
       device='cuda:0')


In [27]:
distilbert_embeddings = distilbert.encode(tweets, convert_to_tensor=True, show_progress_bar=True)
print("\nDistilbert embeddings of tweets:")
print(distilbert_embeddings)

Batches:   0%|          | 0/570 [00:00<?, ?it/s]


Distilbert embeddings of tweets:
tensor([[ 0.4675, -0.1355, -0.0498,  ..., -0.1733, -0.0549, -0.1709],
        [-0.1315,  0.3227, -0.2666,  ...,  0.2122,  0.0090,  0.2963],
        [ 0.1178, -0.2804,  0.6659,  ...,  0.1507,  0.1194, -0.0470],
        ...,
        [ 0.2252, -0.2574,  0.8733,  ...,  0.3147,  0.2412,  0.1965],
        [ 0.3611, -0.0964,  0.0480,  ...,  0.1444, -0.0647,  0.3485],
        [ 0.5175, -0.2305,  0.0106,  ..., -0.1299, -0.1802,  0.3758]],
       device='cuda:0')


In [28]:
minilm_embeddings = para_minilm.encode(tweets, convert_to_tensor=True, show_progress_bar=True)
print("\nPara MiniLM embeddings of tweets:")
print(minilm_embeddings)

Batches:   0%|          | 0/570 [00:00<?, ?it/s]


Para MiniLM embeddings of tweets:
tensor([[-0.2381, -0.2632, -0.2126,  ..., -0.2538,  0.0603,  0.1494],
        [-0.2704,  0.2739,  0.1388,  ..., -0.2232,  0.0223,  0.3507],
        [-0.1678, -0.0078,  0.0854,  ...,  0.0861, -0.1443,  0.4537],
        ...,
        [ 0.0109, -0.1098,  0.0752,  ..., -0.1349, -0.0610, -0.0677],
        [-0.2947, -0.1220, -0.2782,  ...,  0.2311,  0.4245, -0.0535],
        [-0.3133,  0.0155, -0.2157,  ...,  0.2083,  0.3743,  0.0151]],
       device='cuda:0')


In [29]:
xlm_ngram_embeddings = xlm_dist.encode(unique_ngrams, convert_to_tensor=True, show_progress_bar=True)
print("\nXLM-Roberta embeddings of n-grams:")
print(xlm_ngram_embeddings)

Batches:   0%|          | 0/3821 [00:00<?, ?it/s]


XLM-Roberta embeddings of n-grams:
tensor([[ 0.0775, -0.1762,  0.5333,  ..., -0.2868, -0.0343,  0.1394],
        [ 0.1751,  0.0016,  0.1592,  ...,  0.3116, -0.1868, -0.1607],
        [ 0.0920,  0.0666,  0.2239,  ..., -0.7173, -0.0380,  0.1172],
        ...,
        [-0.0498, -0.1296,  0.1285,  ..., -0.1585, -0.0898,  0.0574],
        [-0.0515, -0.0272,  0.2836,  ..., -0.3093,  0.1377,  0.0809],
        [ 0.0824, -0.0169,  0.1811,  ...,  0.1972, -0.1185,  0.1396]],
       device='cuda:0')


In [30]:
distilbert_ngram_embeddings = distilbert.encode(unique_ngrams, convert_to_tensor=True, show_progress_bar=True)
print("\nDistilbert embeddings of n-grams:")
print(distilbert_ngram_embeddings)

Batches:   0%|          | 0/3821 [00:00<?, ?it/s]


Distilbert embeddings of n-grams:
tensor([[-0.0600, -0.0321,  0.1510,  ..., -0.4547,  0.3320, -0.1586],
        [ 0.5079, -0.2232,  0.1212,  ..., -0.1048, -0.5498, -0.0170],
        [-0.0639, -0.0698, -0.2633,  ...,  0.2630, -0.4351,  0.3983],
        ...,
        [-0.3676, -0.3565, -0.0344,  ..., -0.0840, -0.0417, -0.0590],
        [-0.1709, -0.1904,  0.1894,  ..., -0.3434, -0.0365, -0.1402],
        [ 0.3434,  0.1566,  0.2818,  ...,  0.0865, -0.4317,  0.2092]],
       device='cuda:0')


In [31]:
minilm_ngram_embeddings = para_minilm.encode(unique_ngrams, convert_to_tensor=True, show_progress_bar=True)
print("\nPara MiniLM embeddings of n-grams:")
print(minilm_ngram_embeddings)

Batches:   0%|          | 0/3821 [00:00<?, ?it/s]


Para MiniLM embeddings of n-grams:
tensor([[ 0.3074,  0.2692, -0.3288,  ..., -0.8237,  0.1173,  0.3028],
        [-0.2339, -0.0931, -0.3174,  ..., -0.5069,  0.1611,  0.3131],
        [ 0.2517,  0.2675, -0.0580,  ..., -0.0059,  0.9115,  0.4154],
        ...,
        [ 0.2105,  0.2681, -0.4704,  ..., -0.0480,  1.0659,  0.4670],
        [ 0.1358, -0.0122, -0.4407,  ..., -0.4115, -0.3101,  0.1007],
        [-0.6100, -0.2162, -0.2406,  ..., -0.2285,  0.5098,  0.3168]],
       device='cuda:0')


In [32]:
torch.save(xlm_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/xlm_tweet_embeddings.pt")
torch.save(xlm_ngram_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/xlm_ngram_embeddings.pt")
torch.save(distilbert_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/distilbert_tweet_embeddings.pt")
torch.save(distilbert_ngram_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/distilbert_ngram_embeddings.pt")
torch.save(minilm_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/minilm_tweet_embeddings.pt")
torch.save(minilm_ngram_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/minilm_ngram_embeddings.pt")

## Computing average Cosine Similarity between each tri-gram and the document

In [33]:
xlm_similarities = []
for ngram in xlm_ngram_embeddings:
    xlm_similarities.append(util.pytorch_cos_sim(xlm_embeddings, ngram).mean().item())

In [34]:
xlm_similarities

[0.08781898766756058,
 0.1099873036146164,
 0.08273777365684509,
 0.273209810256958,
 0.06145612522959709,
 0.2523340880870819,
 0.043663930147886276,
 0.15908464789390564,
 0.16478365659713745,
 0.08011791110038757,
 0.09882612526416779,
 0.0997481495141983,
 0.04534941539168358,
 0.036307260394096375,
 0.3203964829444885,
 0.04388986527919769,
 0.12823419272899628,
 0.026290973648428917,
 0.05399618297815323,
 0.05715508013963699,
 0.15820874273777008,
 0.15952691435813904,
 0.11441910266876221,
 0.1125696673989296,
 0.00615531112998724,
 0.08319912105798721,
 0.1726500540971756,
 0.054713502526283264,
 0.02401808276772499,
 0.07639678567647934,
 0.0742826759815216,
 0.008432483300566673,
 0.0017163795419037342,
 0.19827987253665924,
 0.01140524074435234,
 0.14766819775104523,
 0.35467812418937683,
 0.15338201820850372,
 0.20240873098373413,
 0.12069298326969147,
 0.004803576041013002,
 0.27053919434547424,
 0.08015836030244827,
 0.09679031372070312,
 0.10056459903717041,
 0.15145118

In [35]:
# minilm_similarity_matrix = util.pytorch_cos_sim(minilm_embeddings, minilm_ngram_embeddings)
# minilm_similarity_matrix

minilm_similarities = []
for ngram in minilm_ngram_embeddings:
    minilm_similarities.append(util.pytorch_cos_sim(minilm_embeddings, ngram).mean().item())

In [36]:
minilm_similarities

[0.15893298387527466,
 0.15371498465538025,
 0.13108199834823608,
 0.3282918930053711,
 0.08514633774757385,
 0.3200676143169403,
 0.15873749554157257,
 0.16459651291370392,
 0.2571077048778534,
 0.12174110114574432,
 0.055979058146476746,
 0.15404146909713745,
 0.19336119294166565,
 0.0612001046538353,
 0.2733316123485565,
 0.094525046646595,
 0.16374215483665466,
 0.1896997094154358,
 0.0846410021185875,
 0.15074637532234192,
 0.24929186701774597,
 0.1372719258069992,
 0.23600724339485168,
 0.17413541674613953,
 0.00580005906522274,
 0.11262209713459015,
 0.17454952001571655,
 0.007287618704140186,
 0.042945753782987595,
 0.15045253932476044,
 -0.024653436616063118,
 0.07064238935709,
 0.06230893358588219,
 0.34552863240242004,
 0.04972897097468376,
 0.23590168356895447,
 0.22835217416286469,
 0.1272645890712738,
 0.28650325536727905,
 0.16530488431453705,
 0.10968467593193054,
 0.2966064512729645,
 0.11087000370025635,
 0.15842828154563904,
 0.18397535383701324,
 0.3067095875740051,

In [37]:
# minilm_similarity_matrix.shape

In [38]:
# # distilbert_similarity_matrix = util.pytorch_cos_sim(distilbert_embeddings, distilbert_ngram_embeddings)
# from sklearn.metrics.pairwise import cosine_similarity
# distilbert_embeddings = distilbert_embeddings.to("cpu")
# distilbert_ngram_embeddings = distilbert_ngram_embeddings.to("cpu")
# distilbert_similarity_matrix = util.pytorch_cos_sim(distilbert_embeddings, distilbert_ngram_embeddings)
# distilbert_similarity_matrix

distilbert_similarities = []
for ngram in distilbert_ngram_embeddings:
    distilbert_similarities.append(util.pytorch_cos_sim(distilbert_embeddings, ngram).mean().item())

In [39]:
# distilbert_similarity_matrix.shape

In [40]:
distilbert_similarities

[0.6800051927566528,
 0.649030327796936,
 0.618944525718689,
 0.7655631303787231,
 0.6296513676643372,
 0.6931222081184387,
 0.7155677676200867,
 0.656707227230072,
 0.6739091873168945,
 0.6388890743255615,
 0.6491064429283142,
 0.6559504270553589,
 0.6693825125694275,
 0.6463773250579834,
 0.7424300312995911,
 0.644248902797699,
 0.6506341099739075,
 0.6619383096694946,
 0.6382203102111816,
 0.6776313781738281,
 0.7038580775260925,
 0.6718186140060425,
 0.6876143217086792,
 0.6695644855499268,
 0.6340504288673401,
 0.7235445976257324,
 0.6750245690345764,
 0.6274701356887817,
 0.643894374370575,
 0.7011646032333374,
 0.62672358751297,
 0.6376720666885376,
 0.6365521550178528,
 0.6823022961616516,
 0.6418787837028503,
 0.6548757553100586,
 0.740950882434845,
 0.643688976764679,
 0.6851047873497009,
 0.6562246680259705,
 0.6612279415130615,
 0.6868333220481873,
 0.6307569742202759,
 0.6559413075447083,
 0.6969977617263794,
 0.6876965165138245,
 0.6837479472160339,
 0.6646904945373535,
 

In [41]:
xlm_similarities = np.array(xlm_similarities)
distilbert_similarities = np.array(distilbert_similarities)
minilm_similarities = np.array(minilm_similarities)

## Normailizing the similarities using MinMaxScaler

In [42]:
scaler = MinMaxScaler()
xlm_similarities = scaler.fit_transform(xlm_similarities.reshape(-1, 1))
distilbert_similarities = scaler.fit_transform(distilbert_similarities.reshape(-1, 1))
minilm_similarities = scaler.fit_transform(minilm_similarities.reshape(-1, 1))

## Putting keyphrases and scores in a dataframe and ranking them individually for each model

In [43]:
xlm_scores_df = pd.DataFrame(list(zip(unique_ngrams, xlm_similarities)), columns=['keyphrase', 'score'])
distilbert_scores_df = pd.DataFrame(list(zip(unique_ngrams, distilbert_similarities)), columns=['keyphrase', 'score'])
minilm_scores_df = pd.DataFrame(list(zip(unique_ngrams, minilm_similarities)), columns=['keyphrase', 'score'])

In [44]:
xlm_scores_df

,keyphrase,score
0,the world in,[0.269510241456006]
1,demand it to,[0.3065532129094173]
2,just clear the,[0.2610195980531811]
3,via nep2020 nepforbharat,[0.5792959268145529]
4,do not deserve,[0.22545823664643955]
...,...,...
122236,from north of,[0.17532264373966733]
122237,under nep2020 they,[0.7337877419699659]
122238,it forward from,[0.2961951456216762]
122239,kamal ghanshala ’s,[0.2592022100131772]


In [45]:
remove_list = []
threshold = 0.3

anomalies1 = xlm_scores_df[xlm_scores_df['score'] < threshold]['keyphrase'].tolist()
remove_list.append(anomalies1)

anomalies2 = distilbert_scores_df[distilbert_scores_df['score'] < threshold]['keyphrase'].tolist()
remove_list.append(anomalies2)

anomalies3 = minilm_scores_df[minilm_scores_df['score'] < threshold]['keyphrase'].tolist()
remove_list.append(anomalies3)

remove_list = [item for sublist in remove_list for item in sublist]
remove_list = list(set(remove_list))

len(remove_list)

68026

In [46]:
# removing the keyphrases from remove_list from all 3 dataframes
xlm_scores_df = xlm_scores_df[~xlm_scores_df['keyphrase'].isin(remove_list)]
distilbert_scores_df = distilbert_scores_df[~distilbert_scores_df['keyphrase'].isin(remove_list)]
minilm_scores_df = minilm_scores_df[~minilm_scores_df['keyphrase'].isin(remove_list)]

In [47]:
xlm_rank_df = xlm_scores_df.sort_values(by='score', ascending=False).reset_index(drop=True)
xlm_rank_df['rank'] = xlm_rank_df.index + 1
xlm_rank_df

,keyphrase,score,rank
0,universityprofrckuhad educationpolicy2020 nept...,[1.0],1
1,education neptransformingindia nep2020,[0.9964947269950803],2
2,education nep2020 neptransformingindia,[0.9931856632252487],3
3,education vakeelsaab nep2020,[0.9864513041234801],4
4,pmnarendramodi nep2020 education,[0.9845888103690903],5
...,...,...,...
54210,moe attitude thursdaymorne,[0.30001674376905263],54211
54211,dispensation as well,[0.30001477669677196],54212
54212,mainstream prevent from,[0.3000086887325614],54213
54213,informative webinar thank,[0.3000031485606316],54214


In [48]:
distilbert_rank_df = distilbert_scores_df.sort_values(by='score', ascending=False).reset_index(drop=True)
distilbert_rank_df['rank'] = distilbert_rank_df.index + 1
distilbert_rank_df

,keyphrase,score,rank
0,nep2020 indian educationpolicy2020,[1.0],1
1,nep2020 india educationpolicy2020,[0.9970753113284196],2
2,nep2020 educationpolicy2020 nationaleducationp...,[0.9967582280376379],3
3,india nep2020 educationpolicy2020,[0.9919738887147154],4
4,nep2020 hindi educationpolicy,[0.9910116885185141],5
...,...,...,...
54210,necessary for develop,[0.30014218757476296],54211
54211,aug 4 5,[0.30002316231546056],54212
54212,and change path,[0.3000195915576813],54213
54213,will change ये,[0.30001316419367896],54214


In [49]:
minilm_rank_df = minilm_scores_df.sort_values(by='score', ascending=False).reset_index(drop=True)
minilm_rank_df['rank'] = minilm_rank_df.index + 1
minilm_rank_df

,keyphrase,score,rank
0,education say nep2020,[1.0],1
1,nep2020 education should,[0.999743960359539],2
2,nep2020 reform education,[0.9995253946460865],3
3,education nep2020 change,[0.9986752773638019],4
4,education nep2020 policy,[0.995730773207359],5
...,...,...,...
54210,shri narendramodi share,[0.30019253437279053],54211
54211,age streanh neolardadavinci,[0.30017129834263345],54212
54212,various recognize high,[0.30011423028433193],54213
54213,watch pmmodi address,[0.30010045523577644],54214


In [50]:
# temp1 = xlm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
# temp2 = distilbert_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
# temp3 = minilm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)

# # create a dataframe with n-gram, average score
# avg_score_df = pd.DataFrame(list(zip(temp1['keyphrase'], (temp1['score'] + temp2['score'] + temp3['score'])/3)), columns=['keyphrase', 'avg_score'])
# avg_score_df

In [51]:
# avg_score_df = avg_score_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)
# avg_score_df

In [52]:
# Computing threshold

# avg_scores = avg_score_df['avg_score'].tolist()

# dk = 0
# for i in range(len(avg_scores)-1):
#     dk += abs(avg_scores[i+1] - avg_scores[i])

# dk = dk/(len(avg_scores)-1)

# sk = 0
# for i in range(len(avg_scores)-1):
#     sk += abs(abs(avg_scores[i+1] - avg_scores[i]) - dk)

# sk = sk/(len(avg_scores)-1)
# sk = np.sqrt(sk)

# threshold = sk/dk
# threshold = threshold[0]

# print("d(k) = ", dk)
# print("s(k) = ", sk)
# print("Threshold = ", threshold)

In [53]:
# threshold = 0.5

In [54]:
# anomalies = avg_score_df[avg_score_df['avg_score'] < threshold]['keyphrase'].tolist()
# anomalies

In [55]:
# remove these keyphrases from xlm_rank_df, distilbert_rank_df and minilm_rand_df
# xlm_rank_df = xlm_rank_df[~xlm_rank_df['keyphrase'].isin(anomalies)]
# distilbert_rank_df = distilbert_rank_df[~distilbert_rank_df['keyphrase'].isin(anomalies)]
# minilm_rank_df = minilm_rank_df[~minilm_rank_df['keyphrase'].isin(anomalies)]

## Processing to get the array to perform Rank Aggregation

In [56]:
# sort xlm_rank_df, distilbert_rank_df, minilm_rank_df by alphabetical order of n-grams
temp1 = xlm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp2 = distilbert_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp3 = minilm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)

# form a matrix where columns are the keyphrases and three rows, each row representing the rank of the keyphrase in each model
keyphrase_matrix = pd.DataFrame(columns=['xlm_rank', 'distilbert_rank', 'minilm_rank'])
keyphrase_matrix['xlm_rank'] = temp1['rank']
keyphrase_matrix['distilbert_rank'] = temp2['rank']
keyphrase_matrix['minilm_rank'] = temp3['rank']

m = keyphrase_matrix.to_numpy()
m = m.T
m

array([[ 7734, 44532, 25439, ..., 39233, 42976, 48353],
       [ 9732, 36380, 24055, ..., 39453, 19329, 18651],
       [12670, 54022, 40391, ..., 53936, 40942, 49719]])

In [57]:
m.shape

(3, 54215)

## Rank Aggregation using `ranky` to get the final rank for each keyphrase

In [58]:
# agg_ranks = rk.kemeny_young(m)
# agg_ranks

In [59]:
temp1 = xlm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp2 = distilbert_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp3 = minilm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)

# create a dataframe with n-gram, average score
# avg_score_df = pd.DataFrame(list(zip(temp1['keyphrase'], (temp1['score'] * 0.45 + temp2['score'] * 0.3 + temp3['score'] * 0.25))), columns=['keyphrase', 'avg_score'])
avg_score_df = pd.DataFrame(list(zip(temp1['keyphrase'], (temp1['score'] + temp2['score'] + temp3['score']) / 3)), columns=['keyphrase', 'avg_score'])
avg_score_df = avg_score_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)
avg_score_df

,keyphrase,avg_score
0,nep2020 indian educationpolicy2020,[0.9720943144565691]
1,nep2020 india educationpolicy2020,[0.9696983759120755]
2,education nep2020 neptransformingindia,[0.9687806038116774]
3,education neptransformingindia nep2020,[0.9676907877960055]
4,nep2020 educationreform indiaeducation,[0.967482865148035]
...,...,...
54210,job come out,[0.30699661755735874]
54211,to germany hope,[0.3060007775953643]
54212,send to immediately,[0.3055464492583944]
54213,continue to mislead,[0.3053783079490506]


In [60]:
final_rank_df = avg_score_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)
final_rank_df['rank'] = final_rank_df.index + 1
final_rank_df

,keyphrase,avg_score,rank
0,nep2020 indian educationpolicy2020,[0.9720943144565691],1
1,nep2020 india educationpolicy2020,[0.9696983759120755],2
2,education nep2020 neptransformingindia,[0.9687806038116774],3
3,education neptransformingindia nep2020,[0.9676907877960055],4
4,nep2020 educationreform indiaeducation,[0.967482865148035],5
...,...,...,...
54210,job come out,[0.30699661755735874],54211
54211,to germany hope,[0.3060007775953643],54212
54212,send to immediately,[0.3055464492583944],54213
54213,continue to mislead,[0.3053783079490506],54214


In [61]:
final_rank_df.head(20)

,keyphrase,avg_score,rank
0,nep2020 indian educationpolicy2020,[0.9720943144565691],1
1,nep2020 india educationpolicy2020,[0.9696983759120755],2
2,education nep2020 neptransformingindia,[0.9687806038116774],3
3,education neptransformingindia nep2020,[0.9676907877960055],4
4,nep2020 educationreform indiaeducation,[0.967482865148035],5
5,nep2020 education neweducationpolicy2020,[0.9672629620693295],6
6,nep2020 education should,[0.9670508405063366],7
7,india nep2020 educationpolicy2020,[0.9627267948746502],8
8,india educationpolicy2020 nep2020,[0.9625319067388448],9
9,nep2020 india educationreform,[0.9603881027021889],10


In [62]:
concatenated_tweets = ' '.join(raw_tweets)
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(concatenated_tweets, keyphrase_ngram_range=(3, 3), top_n=100)
keywords

[('nep2020 india education', 0.7022),
 ('nep2020 indian educationpolicy2020', 0.6988),
 ('nep2020 educationpolicy india', 0.6985),
 ('india education nep2020', 0.6975),
 ('nep2020 indian education', 0.697),
 ('education nep2020 india', 0.6953),
 ('indian education nep2020', 0.6915),
 ('nep2020 india educationpolicy2020', 0.6908),
 ('education india nep2020', 0.6866),
 ('educationpolicy nep2020 india', 0.6854),
 ('21stcentury nep2020 indianeducation', 0.6854),
 ('nep2020 indianeducation discuss', 0.684),
 ('india nep2020 education', 0.6817),
 ('nep2020 national education', 0.6809),
 ('nep2020 national educational', 0.6809),
 ('nep2020 educationpolicy2020 indian', 0.6774),
 ('ugc_india views nep2020', 0.6751),
 ('nep2020 educationreforms indiaeducation', 0.6747),
 ('india educationpolicy2020 nep2020', 0.6745),
 ('nep 2020 indianeducation', 0.6736),
 ('nep2020 india educationsystem', 0.6726),
 ('nep2020 india educationreform', 0.6722),
 ('education nep2020 indiaedufuture', 0.6709),
 ('ind

In [63]:
ground_truth = [x[0] for x in keywords]
ground_truth

['nep2020 india education',
 'nep2020 indian educationpolicy2020',
 'nep2020 educationpolicy india',
 'india education nep2020',
 'nep2020 indian education',
 'education nep2020 india',
 'indian education nep2020',
 'nep2020 india educationpolicy2020',
 'education india nep2020',
 'educationpolicy nep2020 india',
 '21stcentury nep2020 indianeducation',
 'nep2020 indianeducation discuss',
 'india nep2020 education',
 'nep2020 national education',
 'nep2020 national educational',
 'nep2020 educationpolicy2020 indian',
 'ugc_india views nep2020',
 'nep2020 educationreforms indiaeducation',
 'india educationpolicy2020 nep2020',
 'nep 2020 indianeducation',
 'nep2020 india educationsystem',
 'nep2020 india educationreform',
 'education nep2020 indiaedufuture',
 'india nep2020 educationpolicy2020',
 'educationpolicy india nep2020',
 'nep2020 indianeducation education',
 'nep2020 education reforms',
 'nep2020 highereducation india',
 'bjp educationpolicy2020 nep',
 'indiaeducation nep2020 edu

In [64]:
precision = 0

for keyphrase in final_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        precision += 1

print("Proposed Model Precision: ", precision)

Proposed Model Precision:  19


In [65]:
recall = 0

for keyphrase in ground_truth:
    if keyphrase in final_rank_df.head(100)['keyphrase'].tolist():
        recall += 1

print("Proposed Model Recall: ", recall)

Proposed Model Recall:  19


In [66]:
xlm_precision = 0
for keyphrase in xlm_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        xlm_precision += 1

print("xlm-roberto Precision: ", xlm_precision)

distilbert_precision = 0
for keyphrase in distilbert_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        distilbert_precision += 1

print("Distilbert Precision: ", distilbert_precision)

minilm_precision = 0
for keyphrase in minilm_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        minilm_precision += 1

print("Mini LM Precision: ", minilm_precision)

xlm-roberto Precision:  11
Distilbert Precision:  10
Mini LM Precision:  8


In [67]:
xlm_recall = 0
for keyphrase in ground_truth:
    if keyphrase in xlm_rank_df.head(100)['keyphrase'].tolist():
        xlm_recall += 1

print("xlm-roberto Recall: ", xlm_recall)

distilbert_recall = 0
for keyphrase in ground_truth:
    if keyphrase in distilbert_rank_df.head(100)['keyphrase'].tolist():
        distilbert_recall += 1

print("distilbert Recall: ", distilbert_recall)

minilm_recall = 0
for keyphrase in ground_truth:
    if keyphrase in minilm_rank_df.head(100)['keyphrase'].tolist():
        minilm_recall += 1

print("Mini LM Recall: ", minilm_recall)

xlm-roberto Recall:  11
distilbert Recall:  10
Mini LM Recall:  8


In [68]:
import scipy.stats

def rank_correlation(rank1, rank2):
  
    rho, pval = scipy.stats.spearmanr(rank1, rank2)
    return rho

# model_ranking = ["keyphrase1", "keyphrase2", "keyphrase3"]
# ground_truth = ["keyphrase2", "keyphrase1", "keyphrase4"]
similarity = rank_correlation(final_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of proposed model with KeyBERT: ", similarity)

spearman similarity of proposed model with KeyBERT:  0.03890789078907891


In [69]:
xlm_similarity = rank_correlation(xlm_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of xlm-roberto with KeyBERT: ", xlm_similarity)

distilbert_similarity = rank_correlation(distilbert_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of distilbert model with KeyBERT: ", distilbert_similarity)

minilm_similarity = rank_correlation(minilm_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of Mini LM model with KeyBERT: ", minilm_similarity)

spearman similarity of xlm-roberto with KeyBERT:  0.19125112511251124
spearman similarity of distilbert model with KeyBERT:  -0.006480648064806481
spearman similarity of Mini LM model with KeyBERT:  -0.08049204920492048


In [73]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip freeze > requirements.txt